In [1]:
import os

In [2]:
%pwd

'/Users/aayushaggarwal/Desktop/Chicken-disease-project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/aayushaggarwal/Desktop/Chicken-disease-project'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_file:Path
    unzip_dir:Path


In [9]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_file=config.local_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
        

In [11]:
import os
import urllib.request as request
import zipfile
from chicken_disease_classification import logger
from chicken_disease_classification.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename=self.config.local_file
            )
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_file))}")
    
    def extract_zip_file(self):
        '''
        zip_path:str
        Access the zip file and extracts all the information and finally creates
        and store them in a directory
        Return None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_file,'r') as f:
            f.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e




[2024-03-14 06:34:51,643: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-14 06:34:51,644: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-14 06:34:51,645: INFO: common: created directory at: artifacts]
[2024-03-14 06:34:51,646: INFO: common: created directory at: artifacts/data_ingestion]
